<a href="https://colab.research.google.com/github/ghostwalkin/Handwriting-analyser/blob/main/pre-processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
os.chdir('/content/gdrive/MyDrive/Img2Txt')

In [3]:
!pwd

/content/gdrive/MyDrive/Img2Txt


In [ ]:
#!unzip -q /content/gdrive/MyDrive/Img2Txt/parser_text.zip  #renamed the files for convinence sake

In [ ]:
#!unzip -q /content/gdrive/MyDrive/Img2Txt/words_img_dir.zip -x '__MACOSX/*' #renamed the files for convinence sake

In [4]:
f=open('/content/gdrive/MyDrive/Img2Txt/parser.txt')
d=f.read()
f.close()
data=d.split('\n')
data[:10]

['a01-000u-00-00 ok 154 408 768 27 51 AT A',
 'a01-000u-00-01 ok 154 507 766 213 48 NN MOVE',
 'a01-000u-00-02 ok 154 796 764 70 50 TO to',
 'a01-000u-00-03 ok 154 919 757 166 78 VB stop',
 'a01-000u-00-04 ok 154 1185 754 126 61 NPT Mr.',
 'a01-000u-00-05 ok 154 1438 746 382 73 NP Gaitskell',
 'a01-000u-00-06 ok 154 1896 757 173 72 IN from',
 'a01-000u-01-00 ok 156 395 932 441 100 VBG nominating',
 'a01-000u-01-01 ok 156 901 958 147 79 DTI any',
 'a01-000u-01-02 ok 156 1112 958 208 42 AP more']

In [5]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import csv
import pandas as pd

In [6]:
#@title resize image function

def resize_image(image,desired_ratio):
  try:
    height,width=image.shape[:2]
    aspect_ratio=width/height

    desired_width,desired_height=desired_ratio
    target=desired_width/desired_height
#conditions to avoid streching images
    if aspect_ratio > target:
      new_width = desired_width
      new_height = int(new_width / aspect_ratio)
    else:
      new_height = desired_height
      new_width = int(new_height * aspect_ratio)
    resized_image = cv.resize(image, (new_width, new_height))

    padding_width = desired_width - new_width
    padding_height = desired_height - new_height

    top = padding_height // 2
    bottom = padding_height - top
    left = padding_width // 2
    right = padding_width - left

    padded_image = cv.copyMakeBorder(resized_image, top, bottom, left, right, cv.BORDER_CONSTANT, value=(255, 255, 255))

    return padded_image
  except Exception as e:
    print(f"Error resizing image '{image}': {e}")
    return None





In [ ]:
#@title New folder of resized images

output_dir='/content/gdrive/MyDrive/Img2Txt/resized_img'
os.makedirs(output_dir, exist_ok=True)

for d in data:
  ds=d.split(' ')
  filename=ds[0]+'.png'
  filestatus=ds[1]
  label=ds[-1]
  fp= ds[0].split('-')

  filepath='words/{}/{}-{}/{}.png'.format(fp[0],fp[0],fp[1],ds[0])

  row = {'filename': filename, 'filestatus': filestatus, 'label': label, 'filepath': filepath}


  if filestatus=='ok':
    if os.path.isfile(filepath):
      image=cv.imread(filepath)
      desired_ratio=(128,32)
      resized_image=resize_image(image,desired_ratio)

      if resized_image is not None:
                output_path = os.path.join(output_dir, filename)
                cv.imwrite(output_path, resized_image)
                print(f"Processing file: {filepath}")
    else:
      print(f"File does not exist: {filepath}")

